In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
from rich.console import Console
load_dotenv(override=True)

In [ ]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [ ]:
openai = OpenAI()

In [ ]:
def add(a: float, b: float) -> float:
    """Add two numbers"""
    result = a + b
    show(f"[green]Add: {a} + {b} = {result}[/green]")
    return result

def substract(a: float, b: float) -> float:
    """Substract b from a"""
    result = a - b
    show(f"[yellow]Substract: {a} - {b} = {result}[/yellow]")
    return result

def multiply(a: float, b: float) -> float:
    """Multiply two numbers"""
    result = a * b
    show(f"[purple]Multiply: {a} * {b} = {result}[/purple]")
    return result

def divide(a: float, b: float) -> float:
    """Divide two numbers"""
    if b == 0:
        return "Error: division by zero!"
    result = a / b
    show(f"[magenta]Divide: {a} / {b} = {result}[/magenta]")
    return result

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Add two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "substract",
            "description": "Substract the second number from the first number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "The number from which we subtract"},
                    "b": {"type": "number", "description": "The number which we subtract"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two numerbs",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "The first number"},
                    "b": {"type": "number", "description": "The second number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "divide",
            "description": "Divide the first number by the second",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "Dividen"},
                    "b": {"type": "number", "description": "Divisor"}
                },
                "required": ["a", "b"]
            }
        }
    }
]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        tool_function = globals().get(tool_name)
        result = tool_function(**arguments)

        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })

    return results

In [ ]:
def loop(messages):
    done = False
    iteration = 0

    while not done:
        iteration += 1
        show(f"[bold cyan] -- Iteration {iteration} --- [/bold cyan]")

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )

        finish_reason = response.choices[0].finish_reason
        message = response.choices[0].message

        if finish_reason == "tool_calls":
            show(f"[dim] Model calls {len(message.tool_calls)} stes.. [/dim]")

            tool_results = handle_tool_calls(message.tool_calls)

            messages.append(message)
            messages.extend(tool_results)
        else:
            done = True
            show("\n[bold green]=== FINAL ANSWER ===[/bold green]")
            if message.content is None:
                show("[red]ERROR: Model didn't return text answer![/red]")
                show(f"Finish reason: {finish_reason}")
                show(f"Message: {message}")
            else:
                show(message.content)

In [ ]:
system_message = """
You are a helpful math assistant.
Use available tools (add, substract, multiply, divide) for calculates step by step.
After completing all calculations, MUST respond to the user with a text message containing the final result.
Do not end the response without providing the final numerical value.
"""

user_message = "TASK 1: Calculate 3 * (96,1 - 12,5) / 2"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

show("[bold] TASK 1: [/bold]")
loop(messages)

user_message = "I have 100 zlotys. I buy 3 products for 12.50 zlotys each, then I get 10 zlotys back. How much do I have left?"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

show("\n\n" + "="*50)
show("[bold]TASK 2: [/bold]")
loop(messages)